# 第24章 类元编程

## 24.1 类的标准属性

- `cls.__bases__`：由类的基类构成的元组。
- `cls.__qualname__`：类或函数的限定名称，即从模块的全局作用域到类的点分路径。
- `cls.__subclasses__()`：这个方法返回包含类的直接子类的列表，防止在超类和子类之间出现循环引用。返回的列表中是内存里现存的子类，不含尚未导入的模块中的子类。
- `cls.mro()`：构建类时，如果需要获取储存在类属性`__mro__`中的超类元组，会调用这个方法。

## 24.2 type：内置的类工厂函数

In [2]:
class MySuperClass:
    ...
    
class MyMixin:
    ...


MyClass = type('MyClass',
              (MySuperClass, MyMixin),
              {'x': 42, 'x2': lambda self: self.x * 2})

type传入的参数：
- name：class关键字后的标识符。
- bases：类标识符后面圆括号内提供的超类元组，如果class语句没有提供超类，则为`(object, )`。
- dict：属性名称到值的映射。

## 24.3 类工厂函数

需求：构建一个简单的工厂函数，用于创建可变对象的类。在一个宠物店应用程序中，以简单的记录存储狗的数据。

In [9]:
from typing import Union, Any
from collections.abc import Iterable, Iterator

# 别名
FieldNames = Union[str, Iterable[str]]

def record_factory(cls_name: str, field_names: FieldNames) -> type[tuple]:
    
    # 使用属性名构建一个元组
    slots = parse_identifiers(field_names)

    def __init__(self, *args, **kwargs) -> None:
        # 接受位置参数和关键字参数
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self) -> Iterator[Any]:
        # 按照__slots__设定的顺序产出字段值
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self): 
        values = ', '.join(f'{name}={value!r}'
            for name, value in zip(self.__slots__, self))
        cls_name = self.__class__.__name__
        return f'{cls_name}({values})'
    
    # 组建类属性字典
    cls_attrs = dict(
        __slots__=slots,
        __init__=__init__,
        __iter__=__iter__,
        __repr__=__repr__,
    )
    
    # 调用type构造函数，构建新类
    return type(cls_name, (object,), cls_attrs)

In [10]:
def parse_identifiers(names: FieldNames) -> tuple[str, ...]:
    if isinstance(names, str):
        # 把以空格或逗号分隔的names转成字符串列表
        names = names.replace(',', ' ').split()
    if not all(s.isidentifier() for s in names):
        raise ValueError('names must all be valid identifiers')
    return tuple(names)

In [11]:
Dog = record_factory('Dog', 'name weight owner')

In [12]:
rex = Dog('Rex', 30, 'Bob')
rex

Dog(name='Rex', weight=30, owner='Bob')

In [13]:
"{2}'s dog weighs {1}kg".format(*rex)

"Bob's dog weighs 30kg"

## 24.4 Checked类第1版：构建`__init_subclass__`方法

In [14]:
from collections.abc import Callable
from typing import Any, NoReturn, get_type_hints


class Field:
    def __init__(self, name: str, constructor: Callable) -> None:
        # 提示构造函数异常
        if not callable(constructor) or constructor is type(None):
            raise TypeError(f'{name!r} type hint must be callable')
        self.name = name
        self.constructor = constructor

    def __set__(self, instance: Any, value: Any) -> None:
        if value is ...:  
            # 调用无参构造函数
            value = self.constructor()
        else:
            try:
                value = self.constructor(value)  
            except (TypeError, ValueError) as e:  
                type_name = self.constructor.__name__
                msg = f'{value!r} is not compatible with {self.name}:{type_name}'
                raise TypeError(msg) from e
        # 存入__dict__中       
        instance.__dict__[self.name] = value

In [15]:
class Checked:
    @classmethod
    def _fields(cls) -> dict[str, type]: 
        return get_type_hints(cls)

    def __init_subclass__(subclass) -> None:
        # 定义当前类的子类时调用
        super().__init_subclass__()          
        # 构造子类的属性
        for name, constructor in subclass._fields().items():   
            setattr(subclass, name, Field(name, constructor))  

    def __init__(self, **kwargs: Any) -> None:
        # 遍历类的各个name
        for name in self._fields():             
            value = kwargs.pop(name, ...)       
            setattr(self, name, value)          
        if kwargs:
            # 与声明不匹配的名称，提示错误
            self.__flag_unknown_attrs(*kwargs)  

    def __setattr__(self, name: str, value: Any) -> None:
        # 截获一切设置实例属性的操作
        if name in self._fields():              
            cls = self.__class__
            # 如果属性名称是已知的，获取对应的描述符
            descriptor = getattr(cls, name)
            descriptor.__set__(self, value)     
        else:                                   
            self.__flag_unknown_attrs(name)

    def __flag_unknown_attrs(self, *names: str) -> NoReturn:  
        plural = 's' if len(names) > 1 else ''
        extra = ', '.join(f'{name!r}' for name in names)
        cls_name = repr(self.__class__.__name__)
        raise AttributeError(f'{cls_name} object has no attribute{plural} {extra}')

    def _asdict(self) -> dict[str, Any]:
        # 根据子类对象的属性创建字典
        return {
            name: getattr(self, name)
            for name, attr in self.__class__.__dict__.items()
            if isinstance(attr, Field)
        }

    def __repr__(self) -> str:
        kwargs = ', '.join(
            f'{key}={value!r}' for key, value in self._asdict().items()
        )
        return f'{self.__class__.__name__}({kwargs})'

In [16]:
class Movie(Checked):  
    title: str  
    year: int
    box_office: float

In [17]:
movie = Movie(title='The Godfather', year=1972, box_office=137)

In [18]:
movie

Movie(title='The Godfather', year=1972, box_office=137.0)

## 24.5 Checked类第2版：使用类装饰器增强类的功能

改进方案：
- 目标：将Checked类改成类装饰器方式。
- 内容：将Checked类中的方法都从类中移出，将Checked用type组装成类装饰器checked。

In [19]:
def _fields(cls: type) -> dict[str, type]:
    return get_type_hints(cls)

def __init__(self: Any, **kwargs: Any) -> None:
    for name in self._fields():
        value = kwargs.pop(name, ...)
        setattr(self, name, value)
    if kwargs:
        self.__flag_unknown_attrs(*kwargs)

def __setattr__(self: Any, name: str, value: Any) -> None:
    if name in self._fields():
        cls = self.__class__
        descriptor = getattr(cls, name)
        descriptor.__set__(self, value)
    else:
        self.__flag_unknown_attrs(name)

def __flag_unknown_attrs(self: Any, *names: str) -> NoReturn:
    plural = 's' if len(names) > 1 else ''
    extra = ', '.join(f'{name!r}' for name in names)
    cls_name = repr(self.__class__.__name__)
    raise AttributeError(f'{cls_name} has no attribute{plural} {extra}')

def _asdict(self: Any) -> dict[str, Any]:
    return {
        name: getattr(self, name)
        for name, attr in self.__class__.__dict__.items()
        if isinstance(attr, Field)
    }

def __repr__(self: Any) -> str:
    kwargs = ', '.join(f'{key}={value!r}' for key, value in self._asdict().items())
    return f'{self.__class__.__name__}({kwargs})'

In [21]:
def checked(cls: type) -> type:
    # 遍历所有属性，构造子类的属性
    for name, constructor in _fields(cls).items():    
        setattr(cls, name, Field(name, constructor))  

    cls._fields = classmethod(_fields)  # type: ignore  
    
    # 被装饰的类的实例方法的模块级函数
    instance_methods = (
        __init__,
        __repr__,
        __setattr__,
        _asdict,
        __flag_unknown_attrs,
    )
    # 将各个函数添加到cls中
    for method in instance_methods:  
        setattr(cls, method.__name__, method)

    return cls 

In [22]:
@checked
class Movie:
    title: str  
    year: int
    box_office: float

In [23]:
movie = Movie(title='The Godfather', year=1972, box_office=137)
movie

Movie(title='The Godfather', year=1972, box_office=137.0)

## 24.6 导入时和运行时比较

在导入时，解释器的执行操作：
1. 从上到下一次性解析完`.py`模块的源码，此时可能抛出SyntaxError。
2. 编译生成用于执行的字节码。
3. 执行编译后的模块中的顶层代码。

## 24.7 元类

### 24.7.1 元类概念

元类是制造类的工厂。type是大多数内置的类和用户定义的类的元类。

In [27]:
class SuperKlass:
    ...
    
class MetaKlass(type):
    ...
    
class Klass(SuperKlass, metaclass=MetaKlass):
    x = 42
    def __init__(self, y):
        self.y = y

类初始化的执行步骤：
1. 实现`MetaKlass.__new__`，可以对参数进行审查和修改，然后传给`super().__new__`。最终调用`type.__new__`创建新的类对象。
2. `super().__new__`返回之后，进一步处理新创建的类。
3. 调用`SuperKlass.__init_subclass__`，传入新创建的类，如果有类装饰器，会应用类装饰器。
4. Python把类对象绑定给所在命名空间中的名称（class语句是顶层语句时，所在命名空间通常是模块全局命名空间）。

### 24.7.2 Bunch元类示例

In [28]:
class MetaBunch(type): 
    # 被当作类方法使用
    def __new__(meta_cls, cls_name, bases, cls_dict):
        # 存放属性名称到默认值的映射
        defaults = {}

        def __init__(self, **kwargs):
            # 把响应的示例属性设为从kwargs中取出的值或默认值
            for name, default in defaults.items():
                setattr(self, name, kwargs.pop(name, default))
            if kwargs:
                extra = ', '.join(kwargs)
                raise AttributeError(f'No slots left for: {extra!r}')

        def __repr__(self):
            rep = ', '.join(f'{name}={value!r}'
                            for name, default in defaults.items()
                            if (value := getattr(self, name)) != default)
            return f'{cls_name}({rep})'
        
        # 初始化新类的命名空间
        new_dict = dict(__slots__=[], __init__=__init__, __repr__=__repr__)
        
        # 迭代用户定义的类的命名空间
        for name, value in cls_dict.items():
            # 如果带双下划线的名称，把对应的项复制到新类的命名空间中
            if name.startswith('__') and name.endswith('__'):  
                if name in new_dict:
                    raise AttributeError(f"Can't set {name!r} in {cls_name!r}")
                new_dict[name] = value
            else:
                # 如果不是，则追加到__slots__中
                new_dict['__slots__'].append(name)
                defaults[name] = value
        # 构建并返回新类        
        return super().__new__(meta_cls, cls_name, bases, new_dict)  


class Bunch(metaclass=MetaBunch):  # <13>
    pass

In [29]:
class Point(Bunch):
    x = 0.0
    y = 0.0
    color = 'gray'

In [30]:
Point(x=1.2, y=3, color='green')

Point(x=1.2, y=3, color='green')

In [31]:
p = Point()
p.x, p.y, p.color

(0.0, 0.0, 'gray')

## 24.8 Checked类第3版：使用元类实现

改进方案：
1. 添加一个空的`__slots__`属性，并告诉`CheckMeta.__new__`，这个类不需要特殊处理。
2. 删除`__init_subclass__`，相关工作现在交给`CheckMeta.__new__`。
3. 删除`__setattr__`。

In [35]:
from collections.abc import Callable
from typing import Any, NoReturn, get_type_hints

class Field:
    def __init__(self, name: str, constructor: Callable) -> None:
        if not callable(constructor) or constructor is type(None):
            raise TypeError(f'{name!r} type hint must be callable')
        self.name = name
        # 根据name参数得到带单下划线的storage_name
        self.storage_name = '_' + name 
        self.constructor = constructor

    def __get__(self, instance, owner=None):
        if instance is None:
            return self
        # 返回存储在名为storage_name的属性中的值
        return getattr(instance, self.storage_name)

    def __set__(self, instance: Any, value: Any) -> None:
        if value is ...:
            value = self.constructor()
        else:
            try:
                value = self.constructor(value)
            except (TypeError, ValueError) as e:
                type_name = self.constructor.__name__
                msg = f'{value!r} is not compatible with {self.name}:{type_name}'
                raise TypeError(msg) from e
        # 使用setattr设置或更新托管属性        
        setattr(instance, self.storage_name, value)  

In [36]:
class CheckedMeta(type):

    def __new__(meta_cls, cls_name, bases, cls_dict):
        if '__slots__' not in cls_dict:
            # 仅当类的cls_dict不含__slots__，增强类的功能
            slots = []
            # 得到所有带注解的属性
            type_hints = cls_dict.get('__annotations__', {})
            
            for name, constructor in type_hints.items():
                # 为每个带注解的属性构建一个Field示例
                field = Field(name, constructor)
                # 覆盖cls_dict中相应的项
                cls_dict[name] = field 
                # 把字段的storage_name追加到一个列表中
                slots.append(field.storage_name)  
            # 填充cls_dict中的__slots__项
            cls_dict['__slots__'] = slots
    
        return super().__new__(
                meta_cls, cls_name, bases, cls_dict)  # <9>

In [37]:
class Checked(metaclass=CheckedMeta):
    __slots__ = ()  # 跳过CheckedMeta.__new__的处理

    @classmethod
    def _fields(cls) -> dict[str, type]:
        return get_type_hints(cls)

    def __init__(self, **kwargs: Any) -> None:
        for name in self._fields():
            value = kwargs.pop(name, ...)
            setattr(self, name, value)
        if kwargs:
            self.__flag_unknown_attrs(*kwargs)

    def __flag_unknown_attrs(self, *names: str) -> NoReturn:
        plural = 's' if len(names) > 1 else ''
        extra = ', '.join(f'{name!r}' for name in names)
        cls_name = repr(self.__class__.__name__)
        raise AttributeError(f'{cls_name} object has no attribute{plural} {extra}')

    def _asdict(self) -> dict[str, Any]:
        return {
            name: getattr(self, name)
            for name, attr in self.__class__.__dict__.items()
            if isinstance(attr, Field)
        }

    def __repr__(self) -> str:
        kwargs = ', '.join(
            f'{key}={value!r}' for key, value in self._asdict().items()
        )
        return f'{self.__class__.__name__}({kwargs})'

In [38]:
class Movie(Checked):  
    title: str  
    year: int
    box_office: float      

In [39]:
movie = Movie(title='The Godfather', year=1972, box_office=137)
movie

Movie(title='The Godfather', year=1972, box_office=137.0)

## 24.9 元类的实际运用

可简化或代替元类的现代功能：
- 类装饰器：比元类更易于理解，而且导致基类与元类产生冲突的可能性更小。
- `__set_name__`：无须自定义元类逻辑，就能自动设置描述符的名称。
- `__init_subclass__`：提供一种自定义类创建过程的方式，比装饰器更简单，但是复杂的类层次结构可能产生冲突。
- 内置的`dict`保留键的插入顺序：不使用`__prepare__`。

元类、类装饰器和`__init_subclass__`的用途：
- 注册子类。
- 验证子类结构。
- 把装饰器一次性应用到多个方法上。
- 序列化对象。
- 映射对象关系。
- 持久存储对象。
- 在类层级实现特殊方法。
- 实现其他语言特有的功能，例如面向方面程序设计。

## 24.10 杂谈

计算机科学的教学方法分为两个流派：
1. 保守派：计算机程序已经变得极其大而复杂，超过了人类思维所能承载的限度。因此，计算机科学教育的任务是**训练平庸的程序员**，这样500个人合作便能开发出恰好满足需求的程序。
2. 激进派：计算机程序已经变得极其大而复杂，超过了人类思维所能承载的限度。因此，计算机科学教育的任务是**教人如何拓展思维，打破常规，学习以更广博、更强大和更灵活的方式思考，让思维超越程序**。编程思想的各个方面在程序中必会得到充分体现。

编程语言的激进：
- Java一开始使用的就是存取方法，编写程序时，可以先把属性设为公开的（遵循KISS原则），公开的属性无需大幅改动，随时都能变成特性。
- 把函数当作一等对象，描述符和高阶函数合在一起思想，使得函数和方法的统一成为可能。
- Python中的类也是一等对象，提供类构建器、类装饰器、以及允许用户定义功能完整的元类。